In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import pandas as pd
import os
import math

class MyModel_0(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_0, self).__init__()
        self.liner1=nn.Linear(1504,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output
    
class MyModel_1(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_1, self).__init__()
        self.liner1=nn.Linear(1504,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output

class CustomLoss(torch.nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output, target):
        loss = torch.mean(0.5*(output - target)**2)   # 定义自己的损失函数，这里是MSE
        return loss
model_0=MyModel_0()#建立模型
model_1=MyModel_1()#建立模型
loss_fn=CustomLoss()#构建损失函数
optimizer_0=optim.Adam(model_0.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
optimizer_1=optim.Adam(model_1.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
B=5000
Epoch=5000
danwei_b=B/(1241*30)
experience=[]
nt=[0,0,0,0]
loss_abc=[]
rootpath='C:\\Users\\98306\\Desktop\\实验\\训练数据\\500'
for i in range(Epoch):
    print('epoch:',i)
    idx=random.randrange(0,100)
    roadpath=str(idx)+'_road.xlsx'
    roadpath=os.path.join(rootpath,roadpath)
    road=pd.read_excel(roadpath)
    road=road.to_numpy()
    
    bpath=str(idx)+'_b.xlsx'
    bpath=os.path.join(rootpath,bpath)
    b=pd.read_excel(bpath,header=None)
    b=b.to_numpy().flatten()
    
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()

    B_i=B
    res=0#记录每轮最大的TD-error
    total_value=0#每轮获得的总价值
    
    for j in range(road.shape[0]):
        for k in range(road.shape[1]):
            if arr_df[k]==0:
                road[j,k]=0
        if road[j].sum()*danwei_b<b[j]:
            continue
        pi=0
        reward=0
        s= np.array([]).astype(np.double)
        s= np.append(s,1)
        s= np.append(s,j)
        s= np.concatenate((s, road[j]))
        s= np.append(s,b[j])
        s= np.concatenate((s,arr_df))
        s= np.append(s,B_i)
        epsilon=0.1
        lamda=random.uniform(0, 1)
        if lamda<epsilon/2:#随机不选
            pi=0
            reward=0
            nt[0]+=1
        elif lamda<epsilon:#随机选择
            pi=1
            reward=road[j].sum()
            nt[1]+=1
        else:
            with torch.no_grad():
                t0=model_0(torch.Tensor(s))
                t1=model_1(torch.Tensor(s))
                t0=t0.detach().numpy()
                t1=t1.detach().numpy()
                
                if t0[0]<t1[0]:
                    pi=1
                    reward=road[j].sum()
                    nt[2]+=1
                else:
                    pi=0
                    reward=0
                    nt[3]+=1

        if j>=1:#存入经验回放数组
            exper=[]
            exper.append(s_old)
            exper.append(pi_old)
            exper.append(reward_old)
            exper.append(s)
            exper.append(pi)
            experience.append(exper)
#         if j==road.shape[0]-1:
#             exper=[]
#             exper.append(s)
#             exper.append(pi)
#             exper.append(reward)
#             exper.append(np.array([0]))
#             exper.append(0)
#             experience.append(exper)
            
        s_old=s
        pi_old=pi
        reward_old=reward
        
        if pi==1:
            for k in range(road.shape[1]):
                if road[j,k]>0:
                    arr_df[k]-=1
            B_i-=road[j].sum()*danwei_b
            

        
        total_value+=reward
        if len(experience)>=1000:
            experience.pop(0)
        if len(experience)>100:
            idx=random.randrange(0,len(experience))#随机抽取一条经验
            replay_s_old=experience[idx][0]#S
            replay_pi_old=experience[idx][1]#A
            replay_reward=experience[idx][2]#R
            replay_s=experience[idx][3]#S
            replay_pi=experience[idx][4]#A
#             if replay_s.shape[0]==1:
#                 x_train=torch.Tensor(replay_s_old)
#                 real=torch.Tensor(np.array([replay_reward]))
#                 if replay_pi_old==0:
#                     pred=model_0(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_0.zero_grad()
#                     loss.backward()
#                     optimizer_0.step()
#                     res=max(res,loss.item())
#                 else:
#                     pred=model_1(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_1.zero_grad()
#                     loss.backward()
#                     optimizer_1.step()
#                     res=max(res,loss.item())
#             else:
            if replay_s.shape[0]!=1:
                x_train=torch.Tensor(replay_s_old)
                y_train=torch.Tensor(replay_s)
                if replay_pi_old==0 and replay_pi==0:
#                     print("111")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==0 and replay_pi==1:
#                     print("222")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==0:
#                     print("333")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
#                         print(real,torch.Tensor(np.array([replay_reward])),model_0(y_train))
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==1:
#                     print("444")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
        
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
            with torch.no_grad():
                if replay_pi_old==1:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                            elif k==752:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                            else:
                                if param1[0][k] < param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                elif replay_pi_old==0:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()
                            elif k==752:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()
                            else:
                                if param1[0][k] < param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()          
                
    print("Total_value:",total_value)
    print("剩余预算:",B_i)
    print("TD-error",res)
    loss_abc.append(math.sqrt(res))
    print(" ")
print(nt)


epoch: 0
Total_value: 13158.249074108995
剩余预算: 3232.8432616023374
TD-error 1079610.875
 
epoch: 1
Total_value: 13413.234285012719
剩余预算: 3198.5986724398717
TD-error 1258161.375
 
epoch: 2
Total_value: 13666.547910123443
剩余预算: 3164.578577743294
TD-error 1586076.0
 
epoch: 3
Total_value: 11978.678085161308
剩余预算: 3391.259993934823
TD-error 1230890.125
 
epoch: 4
Total_value: 13285.745546862494
剩余预算: 3215.7204476413526
TD-error 622095.25
 
epoch: 5
Total_value: 13430.186969735156
剩余预算: 3196.321921872797
TD-error 2250814.25
 
epoch: 6
Total_value: 13197.999210299882
剩余预算: 3227.5048065673
TD-error 1223480.875
 
epoch: 7
Total_value: 13217.22995948219
剩余预算: 3224.922111270188
TD-error 731572.375
 
epoch: 8
Total_value: 12823.59240454395
剩余预算: 3277.7877512028012
TD-error 1179318.75
 
epoch: 9
Total_value: 11585.920033389199
剩余预算: 3444.0075163323677
TD-error 1994779.25
 
epoch: 10
Total_value: 11512.252787779025
剩余预算: 3453.9010491835834
TD-error 781805.1875
 
epoch: 11
Total_value: 13937.10830551

Total_value: 13038.213588241375
剩余预算: 3248.964062820122
TD-error 9699380.0
 
epoch: 94
Total_value: 11911.252272527685
剩余预算: 3400.3153004931955
TD-error 3543436.5
 
epoch: 95
Total_value: 13773.976212368498
剩余预算: 3150.150925010945
TD-error 5713405.0
 
epoch: 96
Total_value: 13310.19216354373
剩余预算: 3212.437259798049
TD-error 6278426.0
 
epoch: 97
Total_value: 12958.385132725061
剩余预算: 3259.6850479821314
TD-error 3651001.0
 
epoch: 98
Total_value: 13627.581309849897
剩余预算: 3169.8118036731266
TD-error 1247350.75
 
epoch: 99
Total_value: 12727.853325769896
剩余预算: 3290.645537769286
TD-error 8428443.0
 
epoch: 100
Total_value: 11716.967509628043
剩余预算: 3426.4078015541186
TD-error 7062911.5
 
epoch: 101
Total_value: 12102.067179203654
剩余预算: 3374.6888021483137
TD-error 8219494.0
 
epoch: 102
Total_value: 13791.786871319373
剩余预算: 3147.758948251494
TD-error 5477167.5
 
epoch: 103
Total_value: 13783.549261146662
剩余预算: 3148.865261731579
TD-error 5224436.5
 
epoch: 104
Total_value: 12767.268924165712
剩

Total_value: 13608.316056104035
剩余预算: 3172.399132943321
TD-error 9587433.0
 
epoch: 187
Total_value: 13580.694334510235
剩余预算: 3176.1087383145023
TD-error 11428252.0
 
epoch: 188
Total_value: 13392.789606363755
剩余预算: 3201.344398823024
TD-error 14847924.0
 
epoch: 189
Total_value: 13819.501378246883
剩余预算: 3144.0368817825834
TD-error 10799710.0
 
epoch: 190
Total_value: 11718.383893142878
剩余预算: 3426.2175808295906
TD-error 16965864.0
 
epoch: 191
Total_value: 11768.88821299723
剩余预算: 3419.43483575111
TD-error 6789313.0
 
epoch: 192
Total_value: 11602.030868297461
剩余预算: 3441.8438264440683
TD-error 12833932.0
 
epoch: 193
Total_value: 13919.892844294598
剩余预算: 3130.5542782306466
TD-error 13104315.0
 
epoch: 194
Total_value: 13790.733037601402
剩余预算: 3147.9004784311846
TD-error 8553305.0
 
epoch: 195
Total_value: 13716.122062533206
剩余预算: 3157.92075442745
TD-error 12281070.0
 
epoch: 196
Total_value: 13605.932315806052
剩余预算: 3172.7192699696416
TD-error 8460791.0
 
epoch: 197
Total_value: 14259.13

Total_value: 12582.445701776882
剩余预算: 3310.173824633778
TD-error 929066.875
 
epoch: 279
Total_value: 12670.252800025874
剩余预算: 3298.3813053953972
TD-error 1010136.8125
 
epoch: 280
Total_value: 13907.991309743948
剩余预算: 3132.1526578372354
TD-error 7709358.0
 
epoch: 281
Total_value: 13818.101309068714
剩余预算: 3144.224911486877
TD-error 24278128.0
 
epoch: 282
Total_value: 13690.081400245868
剩余预算: 3161.4180230666316
TD-error 390322.375
 
epoch: 283
Total_value: 12540.27502024618
剩余预算: 3315.837359623129
TD-error 19266356.0
 
epoch: 284
Total_value: 13617.91171202322
剩余预算: 3171.1104335182345
TD-error 183226.4375
 
epoch: 285
Total_value: 13640.069781369079
剩余预算: 3168.13459825825
TD-error 18182682.0
 
epoch: 286
Total_value: 11716.967509628043
剩余预算: 3426.4078015541186
TD-error 21424134.0
 
epoch: 287
Total_value: 13778.320804878842
剩余预算: 3149.5674449531507
TD-error 18538570.0
 
epoch: 288
Total_value: 12090.196666754422
剩余预算: 3376.283015477516
TD-error 11318724.0
 
epoch: 289
Total_value: 140

Total_value: 13340.298708998997
剩余预算: 3208.3939418481073
TD-error 20033758.0
 
epoch: 371
Total_value: 13401.965605664825
剩余预算: 3200.112059405746
TD-error 15858106.0
 
epoch: 372
Total_value: 13239.395199332448
剩余预算: 3221.94531300934
TD-error 21584196.0
 
epoch: 373
Total_value: 14241.847657461238
剩余预算: 3087.315651697389
TD-error 20741426.0
 
epoch: 374
Total_value: 13174.692379398308
剩余预算: 3230.634920843633
TD-error 25402000.0
 
epoch: 375
Total_value: 12539.580976537838
剩余预算: 3315.930569898222
TD-error 16465858.0
 
epoch: 376
Total_value: 13183.13035169463
剩余预算: 3229.5016986711466
TD-error 13925939.0
 
epoch: 377
Total_value: 14162.278911715044
剩余预算: 3098.0017577605363
TD-error 17216784.0
 
epoch: 378
Total_value: 13286.919388527796
剩余预算: 3215.562800358877
TD-error 18856402.0
 
epoch: 379
Total_value: 12743.595031895524
剩余预算: 3288.5314219855595
TD-error 15033021.0
 
epoch: 380
Total_value: 13796.05714836101
剩余预算: 3147.185448783103
TD-error 18539474.0
 
epoch: 381
Total_value: 13825.0

...

Total_value: 10659.478119697087
剩余预算: 3568.428939068347
TD-error 17397.318359375
 
epoch: 4640
Total_value: 13476.241867108012
剩余预算: 3190.136735548213
TD-error 19110.125
 
epoch: 4641
Total_value: 11793.219203308137
剩余预算: 3416.167176563506
TD-error 14656.2578125
 
epoch: 4642
Total_value: 13752.238591037174
剩余预算: 3153.070293978355
TD-error 7903.58447265625
 
epoch: 4643
Total_value: 12601.193793526469
剩余预算: 3307.6559503724866
TD-error 22339.283203125
 
epoch: 4644
Total_value: 10666.551209185782
剩余预算: 3567.479021060196
TD-error 14811.8271484375
 
epoch: 4645
Total_value: 13177.778391170068
剩余预算: 3230.22046855089
TD-error 17890.412109375
 
epoch: 4646
Total_value: 13088.791112372603
剩余预算: 3242.1714863856296
TD-error 27534.52734375
 
epoch: 4647
Total_value: 12374.218386504454
剩余预算: 3338.1388145978444
TD-error 25109.87890625
 
epoch: 4648
Total_value: 11697.275586737283
剩余预算: 3429.0524326165346
TD-error 9770.763671875
 
epoch: 4649
Total_value: 10304.747480089301
剩余预算: 3616.069368776618


Total_value: 12916.560334816288
剩余预算: 3265.3021306988603
TD-error 6694.05859375
 
epoch: 4728
Total_value: 9522.814286523406
剩余预算: 3721.083227703009
TD-error 13671.5380859375
 
epoch: 4729
Total_value: 9473.79933210632
剩余预算: 3727.6659505632133
TD-error 18424.875
 
epoch: 4730
Total_value: 11488.360531358227
剩余预算: 3457.1097862801194
TD-error 3865.1279296875
 
epoch: 4731
Total_value: 13427.671207669553
剩余预算: 3196.6597894615165
TD-error 31068.623046875
 
epoch: 4732
Total_value: 13071.32038711359
剩余预算: 3244.5178099498266
TD-error 42737.00390625
 
epoch: 4733
Total_value: 12718.083944010872
剩余预算: 3291.957568625991
TD-error 20017.58203125
 
epoch: 4734
Total_value: 9839.86995657895
剩余预算: 3678.502557537073
TD-error 41691.1015625
 
epoch: 4735
Total_value: 7716.944982789836
剩余预算: 3963.6120087577456
TD-error 26655.240234375
 
epoch: 4736
Total_value: 13116.02162616939
剩余预算: 3238.514420337176
TD-error 13458.572265625
 
epoch: 4737
Total_value: 9483.186063621815
剩余预算: 3726.4053097472724
TD-erro

Total_value: 11077.894144854346
剩余预算: 3512.2355432642566
TD-error 41778.0
 
epoch: 4816
Total_value: 7951.859182393472
剩余预算: 3932.0629623430737
TD-error 13837.720703125
 
epoch: 4817
Total_value: 13486.371869638784
剩余预算: 3188.7762732153133
TD-error 35612.17578125
 
epoch: 4818
Total_value: 12579.400313721795
剩余预算: 3310.5828211493695
TD-error 25317.333984375
 
epoch: 4819
Total_value: 12994.526668676817
剩余预算: 3254.8312290254075
TD-error 91232.0625
 
epoch: 4820
Total_value: 12191.270772750031
剩余预算: 3362.708733178884
TD-error 63592.08984375
 
epoch: 4821
Total_value: 12823.611643350974
剩余预算: 3277.785167425332
TD-error 29717.505859375
 
epoch: 4822
Total_value: 13608.53169164336
剩余预算: 3172.370173026677
TD-error 45915.74609375
 
epoch: 4823
Total_value: 13055.089204023621
剩余预算: 3246.6976626344867
TD-error 34601.96875
 
epoch: 4824
Total_value: 10108.87688027643
剩余预算: 3642.3748482035417
TD-error 40003.0546875
 
epoch: 4825
Total_value: 11041.973371955024
剩余预算: 3517.05971367781
TD-error 2547

Total_value: 12335.688045749672
剩余预算: 3343.31345074541
TD-error 61155.015625
 
epoch: 4904
Total_value: 12935.255055264706
剩余预算: 3262.791424219084
TD-error 34116.59765625
 
epoch: 4905
Total_value: 12816.174073214941
剩余预算: 3278.7840352921116
TD-error 60659.4375
 
epoch: 4906
Total_value: 12898.238229607636
剩余预算: 3267.762794841845
TD-error 52173.96484375
 
epoch: 4907
Total_value: 10256.2689076278
剩余预算: 3622.580055381708
TD-error 20800.4296875
 
epoch: 4908
Total_value: 12981.812401329142
剩余预算: 3256.5387588867666
TD-error 48271.25
 
epoch: 4909
Total_value: 12902.826008561953
剩余预算: 3267.14665477277
TD-error 40106.4296875
 
epoch: 4910
Total_value: 9542.09987862172
剩余预算: 3718.4931669860707
TD-error 46861.04296875
 
epoch: 4911
Total_value: 12906.833535215945
剩余预算: 3266.6084427590736
TD-error 62243.84375
 
epoch: 4912
Total_value: 12736.76465952595
剩余预算: 3289.4487430129006
TD-error 49341.55078125
 
epoch: 4913
Total_value: 13314.544812896278
剩余预算: 3211.852697703966
TD-error 37414.48046875

Total_value: 9494.42132478444
剩余预算: 3724.896410853553
TD-error 24101.2734375
 
epoch: 4993
Total_value: 7056.923698538793
剩余预算: 4052.253062242977
TD-error 3610.342041015625
 
epoch: 4994
Total_value: 12826.411030915979
剩余预算: 3277.409208848243
TD-error 141213.953125
 
epoch: 4995
Total_value: 13356.74907784004
剩余预算: 3206.1846524523176
TD-error 34191.63671875
 
epoch: 4996
Total_value: 9072.561689000202
剩余预算: 3781.552284582298
TD-error 29070.6328125
 
epoch: 4997
Total_value: 12500.881728630784
剩余预算: 3321.1278903262432
TD-error 41293.31640625
 
epoch: 4998
Total_value: 12880.565805524542
剩余预算: 3270.136206617709
TD-error 69475.6484375
 
epoch: 4999
Total_value: 12717.151199209211
剩余预算: 3292.0828365284437
TD-error 134225.75
 
[8462, 8630, 94109, 58433]


In [1]:
rootpath='C:\\Users\\98306\\Desktop\\实验\\训练数据\\500'
for i in range(2000):
    print('epoch:',i)
    idx=random.randrange(0,100)
    roadpath=str(idx)+'_road.xlsx'
    roadpath=os.path.join(rootpath,roadpath)
    road=pd.read_excel(roadpath)
    road=road.to_numpy()
    
    bpath=str(idx)+'_b.xlsx'
    bpath=os.path.join(rootpath,bpath)
    b=pd.read_excel(bpath,header=None)
    b=b.to_numpy().flatten()
    
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()

    B_i=B
    res=0#记录每轮最大的TD-error
    total_value=0#每轮获得的总价值
    
    for j in range(road.shape[0]):
        for k in range(road.shape[1]):
            if arr_df[k]==0:
                road[j,k]=0
        if road[j].sum()*danwei_b<b[j]:
            continue
        pi=0
        reward=0
        s= np.array([]).astype(np.double)
        s= np.append(s,1)
        s= np.append(s,j)
        s= np.concatenate((s, road[j]))
        s= np.append(s,b[j])
        s= np.concatenate((s,arr_df))
        s= np.append(s,B_i)
        epsilon=0.1
        lamda=random.uniform(0, 1)
        if lamda<epsilon/2:#随机不选
            pi=0
            reward=0
            nt[0]+=1
        elif lamda<epsilon:#随机选择
            pi=1
            reward=road[j].sum()
            nt[1]+=1
        else:
            with torch.no_grad():
                t0=model_0(torch.Tensor(s))
                t1=model_1(torch.Tensor(s))
                t0=t0.detach().numpy()
                t1=t1.detach().numpy()
                
                if t0[0]<t1[0]:
                    pi=1
                    reward=road[j].sum()
                    nt[2]+=1
                else:
                    pi=0
                    reward=0
                    nt[3]+=1

        if j>=1:#存入经验回放数组
            exper=[]
            exper.append(s_old)
            exper.append(pi_old)
            exper.append(reward_old)
            exper.append(s)
            exper.append(pi)
            experience.append(exper)
#         if j==road.shape[0]-1:
#             exper=[]
#             exper.append(s)
#             exper.append(pi)
#             exper.append(reward)
#             exper.append(np.array([0]))
#             exper.append(0)
#             experience.append(exper)
            
        s_old=s
        pi_old=pi
        reward_old=reward
        
        if pi==1:
            for k in range(road.shape[1]):
                if road[j,k]>0:
                    arr_df[k]-=1
            B_i-=road[j].sum()*danwei_b
            

        
        total_value+=reward
        if len(experience)>=1000:
            experience.pop(0)
        if len(experience)>100:
            idx=random.randrange(0,len(experience))#随机抽取一条经验
            replay_s_old=experience[idx][0]#S
            replay_pi_old=experience[idx][1]#A
            replay_reward=experience[idx][2]#R
            replay_s=experience[idx][3]#S
            replay_pi=experience[idx][4]#A
#             if replay_s.shape[0]==1:
#                 x_train=torch.Tensor(replay_s_old)
#                 real=torch.Tensor(np.array([replay_reward]))
#                 if replay_pi_old==0:
#                     pred=model_0(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_0.zero_grad()
#                     loss.backward()
#                     optimizer_0.step()
#                     res=max(res,loss.item())
#                 else:
#                     pred=model_1(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_1.zero_grad()
#                     loss.backward()
#                     optimizer_1.step()
#                     res=max(res,loss.item())
#             else:
            if replay_s.shape[0]!=1:
                x_train=torch.Tensor(replay_s_old)
                y_train=torch.Tensor(replay_s)
                if replay_pi_old==0 and replay_pi==0:
#                     print("111")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==0 and replay_pi==1:
#                     print("222")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==0:
#                     print("333")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
#                         print(real,torch.Tensor(np.array([replay_reward])),model_0(y_train))
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==1:
#                     print("444")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
        
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
            with torch.no_grad():
                if replay_pi_old==1:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                            elif k==752:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                            else:
                                if param1[0][k] < param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                elif replay_pi_old==0:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()
                            elif k==752:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()
                            else:
                                if param1[0][k] < param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()          
                
    print("Total_value:",total_value)
    print("剩余预算:",B_i)
    print("TD-error",res)
    loss_abc.append(math.sqrt(res))
    print(" ")
print(nt)

epoch: 0


NameError: name 'random' is not defined